# Setup

In [66]:
# path to the config file (see example). 
# will work with a config file for https://github.com/kimberli/mturk-template, 
# but a minimal example is included.
# Also supports an additional feature, the "variants" key, which is a list of dictionaries. 
# If "variants" is specified, for each dictionary it contains, those keys will be meshed with the "hitCreation"
# key and one task will be made per variant. Else, config["hitCreation"]["numTasks"] versions of the same
# task will be launched. 
CONFIG_PATH = "./config.json"

# where to save downloaded results 
SAVE_PATH = "./result.csv" 

# Whether to launch a hit per fold.txt in folder "files"
LAUNCH_HITS_FOR_ALL_FOLDS = False

## Define folds to use in the hit
The folds are txt files containing paths to the images to be used in the hit. This notebooks allows for creation of multiple hits, each with a different fold.txt .
In the folder "files", multiple txt files (folds) can be found. Each one contains 10 images, and there are no repeats among folds. Each fold has a piece of the full canva_scraping2 dataset.

In [67]:
from boto3 import client
import json
import copy

_USING_PROD = None

with open(CONFIG_PATH, 'r') as f:
    config = json.loads(f.read())
    hit_config = config['hitCreation']

if hit_config['production']:
    print("USING PROD")
    _USING_PROD = True
    endpoint_url = 'https://mturk-requester.us-east-1.amazonaws.com'
else:
    print("USING SANDBOX")
    _USING_PROD = False
    endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'
        
cl = client('mturk', region_name='us-east-1', endpoint_url=endpoint_url)

if hit_config['fold']:
    hit_config['taskUrl'] = hit_config['taskUrl'] + "?url=%s" % hit_config['fold']

if LAUNCH_HITS_FOR_ALL_FOLDS:
    all_folds = os.listdir('./all_folds')        
    
print("TASK URL:", hit_config['taskUrl'])


USING SANDBOX
TASK URL: https://cfosco.github.io/mturk-importance/?url=fold2_dropbox.txt


# Make new HIT

In [68]:
# Safety flags that prevent you from accidentally messing up your HITs. 
# Set to False except when you are performing these specific tasks. 
ALLOW_HIT_CREATION = True
ALLOW_ASSIGNMENT_ADDITION = False
ALLOW_CREATE_QUAL = False
ALLOW_UPDATE_EXPIRATION = False

In [69]:
# List of qualifications that you will use to filter potential workers. 
# These require that workers come from the US and have an approval rating >= 95%
QUALS = [
       {
           'QualificationTypeId': '00000000000000000071',
           'Comparator': 'EqualTo',
           'LocaleValues': [{
               'Country': 'US',
           }],
       },
        
       {
           'QualificationTypeId': '000000000000000000L0',
           'Comparator': 'GreaterThanOrEqualTo',
           'IntegerValues': [
               95
           ],
       },
    ]

In [72]:
# creates a HIT in the form of an External Question inside an iFrame
def create_hit(task):
    questionText = "<ExternalQuestion xmlns=\"http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/"
    questionText += "2006-07-14/ExternalQuestion.xsd\">\n<ExternalURL>" + task['taskUrl']
    questionText += "</ExternalURL>\n  <FrameHeight>700</FrameHeight>\n</ExternalQuestion>"

    response = cl.create_hit(
        MaxAssignments=task['numAssignments'],
        AutoApprovalDelayInSeconds=604800,
        LifetimeInSeconds=task['lifetime'],
        AssignmentDurationInSeconds=task['duration'],
        Reward=task['rewardAmount'],
        Title=task['title'],
        Keywords=task['keywords'],
        Description=task['description'],
        Question=questionText,
        QualificationRequirements=QUALS,
    )

    print(response)
    print("\n")

In [129]:
if ALLOW_HIT_CREATION: 
    if config.get('variants', None): 
        print("creating " + str(len(config['variants'])) + " variants")
        for var in config['variants']: 
            task = copy.deepcopy(config)
            task.update(var)
            create_hit(task)
    
    elif LAUNCH_HITS_FOR_ALL_FOLDS:
        print("creating", len(all_folds), "tasks")
        for fold in all_folds:
            hit_config['taskUrl'] = hit_config['taskUrl'] + "?url=%s" % fold
            create_hit(hit_config)
    else:
        print("creating " + str(hit_config['numTasks']) + " tasks")
        for i in range(hit_config['numTasks']):
            create_hit(hit_config)

creating 1 tasks
{'HIT': {'HITId': '3IH9TRB0FBG6ZNKUDKVO181V2OKI1Q', 'HITTypeId': '3XFGHWX5QHYHQBDGV0YMSL03FYN6JO', 'HITGroupId': '3BMWOOFJV0EY7LXB3KKNMZC069YSTE', 'CreationTime': datetime.datetime(2019, 1, 30, 1, 55, 37, tzinfo=tzlocal()), 'Title': 'Annotate the most important regions on graphic designs', 'Description': 'Manually highlight the important parts of a graphic design. You will be shown a set of 5 images, and you will have to indicate which parts feel important to you.', 'Question': '<ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">\n<ExternalURL>https://cfosco.github.io/mturk-importance/?url=fold2_dropbox.txt</ExternalURL>\n  <FrameHeight>700</FrameHeight>\n</ExternalQuestion>', 'Keywords': 'labeling, importance, highlighting', 'HITStatus': 'Assignable', 'MaxAssignments': 2000, 'Reward': '0.00', 'AutoApprovalDelayInSeconds': 604800, 'Expiration': datetime.datetime(2019, 2, 1, 5, 42, 17, tzinfo=tzloca

# HIT monitoring helpers

In [149]:
MAX_RESULTS = 65 # set equal to the number of outstanding hits you have 

hits = cl.list_hits(MaxResults=MAX_RESULTS)['HITs']

In [150]:
print(len(hits))

51


In [151]:
# Gets all assignments created for a HIT
def get_all_assignments(hitid): 
    assignments = []
    should_continue = True
    next_token = False
    while (should_continue): 
        args = {
            'HITId': hitid, 
            'MaxResults': 100
        }
        if (next_token): 
            args['NextToken'] = next_token
        r = cl.list_assignments_for_hit(**args)
        next_token = r.get('NextToken', False)
        assignments.extend(r["Assignments"])
        should_continue = len(r["Assignments"]) > 0
    return assignments

In [152]:
import datetime

# Summarizes all hits in `hits` in a human-readable way 
def summarize_hits(hits): 
    print(len(hits))
    ret = ""
    for hit in hits: 
        expiration = hit['Expiration'].replace(tzinfo=None)
        is_expired = expiration < datetime.datetime.now()
        description = ("Title: {title}\n" 
        "ID: {hid}\n"
        "\tAssignments left: {left}\n"
        "\tAssignments completed: {complete}\n"
        "\tAssignments pending: {pending}\n"
        "\tExpired: {exp}\n\n").format(
            title=hit['Title'], 
            hid=hit['HITId'], 
            left=hit['NumberOfAssignmentsAvailable'], 
            complete=hit['NumberOfAssignmentsCompleted'], 
            pending=hit['NumberOfAssignmentsPending'],
            exp=str(is_expired)
        )
        ret += description
    print(ret)

In [153]:
# Summarizes all pending/submitted/approved assignments for all hits in `hits`
def summarize_assignments(hits):
    ret = ""
    for hit in hits: 
        hid = hit['HITId']
        title =  hit['Title']
        name = "HIT %s: %s" % (hid, title)
        ret += name + "\n"
        assignments = get_all_assignments(hid)
        for a in assignments: 
            desc = "\tAssignment {aid}\n\t\tStatus: {status}\n".format(aid=a['AssignmentId'], status=a['AssignmentStatus'])
            ret += desc
    print(ret)

In [154]:
def refresh_hits(): 
    global hits 
    global MAX_RESULTS
    hits = cl.list_hits(MaxResults=MAX_RESULTS)['HITs']

# HIT monitoring

In [155]:
refresh_hits()
pass

In [156]:
refresh_hits()

summarize_hits(hits)

51
Title: Annotate the most important regions on graphic designs
ID: 3IH9TRB0FBG6ZNKUDKVO181V2OKI1Q
	Assignments left: 2000
	Assignments completed: 0
	Assignments pending: 0
	Expired: False

Title: Annotate the most important regions on graphic designs
ID: 3909MD9T2ZYL9XCGO0KI5V7XB5WFEF
	Assignments left: 2000
	Assignments completed: 0
	Assignments pending: 0
	Expired: False

Title: Annotate the most important regions on graphic designs
ID: 302U8RURJZIEZ0BJHES5D44SDRKNVZ
	Assignments left: 2000
	Assignments completed: 0
	Assignments pending: 0
	Expired: False

Title: Annotate the most important regions on graphic designs
ID: 3G5RUKN2ECKG2TYGEDN9LMFVMZ19N5
	Assignments left: 2000
	Assignments completed: 0
	Assignments pending: 0
	Expired: False

Title: Annotate the most important regions on graphic designs
ID: 3AJA9FLWSCFXP2E2KJA9QA5P3Y2IFQ
	Assignments left: 2000
	Assignments completed: 0
	Assignments pending: 0
	Expired: False

Title: Annotate the most important regions on graphic des

In [157]:
refresh_hits()
summarize_assignments(hits)
pass

HIT 3IH9TRB0FBG6ZNKUDKVO181V2OKI1Q: Annotate the most important regions on graphic designs
HIT 3909MD9T2ZYL9XCGO0KI5V7XB5WFEF: Annotate the most important regions on graphic designs
HIT 302U8RURJZIEZ0BJHES5D44SDRKNVZ: Annotate the most important regions on graphic designs
HIT 3G5RUKN2ECKG2TYGEDN9LMFVMZ19N5: Annotate the most important regions on graphic designs
HIT 3AJA9FLWSCFXP2E2KJA9QA5P3Y2IFQ: Annotate the most important regions on graphic designs
HIT 3Q7TKIAPOTR2HJ8CEGCWUQUW8PGLDW: Annotate the most important regions on graphic designs
HIT 360ZO6N6J10AKBDG2C8987QVH9R9M5: Annotate the most important regions on graphic designs
HIT 3EHIMLB7F7GOESTV6NMIG6NOGHAH8F: Annotate the most important regions on graphic designs
	Assignment 3VBEN272MLGVJ0BNQHF71H75KOGGSU
		Status: Submitted
HIT 3LCXHSGDLTNUD2H2UKKHCUQC0VPESW: Annotate the most important regions on graphic designs
	Assignment 388U7OUMF8IM7H0PXQQZQFFTR5BR0H
		Status: Submitted
HIT 3CVBMEMMXBXKEEOCWIUC26RT5X4H7S: Annotate the most i

# Approve HITs

In [163]:
# Approves all outstanding hits created for the HITs in hits 
def approve_all(hits): 
    num_approved = 0
    for hit in hits: 
        # make sure you keep getting assignments 
        assignments = get_all_assignments(hit["HITId"])
        #print(assignments)
        for a in assignments: 
            if a['AssignmentStatus'] != 'Approved':
                print("Approving assignment")
                num_approved += 1
                cl.approve_assignment(AssignmentId=a['AssignmentId'])
    print("Approved %d assignments" % num_approved)

In [164]:
refresh_hits()
approve_all(hits)

Approving assignment
Approved 1 assignments


# Update expiration or num tasks

In [116]:
import datetime 

# changes the expiration date on a HIT to days_from_now days in the future
def update_expiration(hitid, days_from_now): 
    if ALLOW_UPDATE_EXPIRATION: 
        days = days_from_now*datetime.timedelta(days=1)
        expire_time = datetime.datetime.now() + days

        response = cl.update_expiration_for_hit(HITId=hitid, ExpireAt=expire_time)
        print(response)
        return response
    else: 
        raise RuntimeException("This action is not currently enabled; set `ALLOW_UPDATE_EXPIRATION` to true to proceed with this action")
    
def expire_hit(hit): 
    return update_expiration(hit, -10)

In [117]:
def add_assignments(hitid, num_assignments): 
    if ALLOW_ASSIGNMENT_ADDITION: 
        response = cl.create_additional_assignments_for_hit(
            HITId=hitid,
            NumberOfAdditionalAssignments=num_assignments
        )
        print(response)
        return response
    else: 
        raise RuntimException("This action is not currently enabled; set `ALLOW_ASSIGNMENT_ADDITION` to true to proceed with this action")

# Add custom qualifications 

## Add a qualification to disqualify workers who have done work before

- uses "negative qualification" method from https://github.com/cloudyr/MturkR/wiki/qualifications-as-blocks

### NOTE: quals are kept separate for the sandbox and prod. Make sure you are creating and assigning your quals in prod. 

In [33]:
# structure of a new qualification 
NEW_QUAL = {
    'Name': 'qualName',
    'Keywords': 'Keywords for qual',
    'Description': 'What is this qual, and why are you assigning it?',
    'QualificationTypeStatus': 'Active',
    'AutoGranted': False
}

In [34]:
def create_qual(new_qual):
    if ALLOW_CREATE_QUAL: 
        response = cl.create_qualification_type(**new_qual)
        print(response)
        Id = response['QualificationTypeId']
        print("id", Id)
        return Id
    else: 
        raise RuntimException("This action is not currently enabled; set `ALLOW_CREATE_QUAL` to true to proceed with this action")

In [38]:
# Gets all the custom quals you have created. 
def list_quals(): 
    response = cl.list_qualification_types(
            Query='hasCompletedVisualGraphRecallTask',
            MustBeRequestable=False
    )
    print(response)

list_quals()

{'NumResults': 0, 'QualificationTypes': [], 'ResponseMetadata': {'RequestId': '4ff9cff9-32f4-49b2-b79a-0137726813e0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4ff9cff9-32f4-49b2-b79a-0137726813e0', 'content-type': 'application/x-amz-json-1.1', 'content-length': '40', 'date': 'Tue, 27 Nov 2018 21:11:39 GMT'}, 'RetryAttempts': 0}}


In [39]:
def assign_qual(qual_id, worker_ids): 
    for worker in worker_ids: 
        response = cl.associate_qualification_with_worker(
                QualificationTypeId=qual_id, 
                WorkerId=worker,
                IntegerValue=1,
                SendNotification=False
        )
        print(response)
        assert response
        
def get_workers_for_hit(hitid): 
    a = get_all_assignments(hitid)
    workers = [a_['WorkerId'] for a_ in a]
    return workers
    
def confirm_quals(qual_id, worker_ids): 
    for w in worker_ids: 
        response = cl.get_qualification_score(
                QualificationTypeId=qual_id,
                WorkerId=w
        )
        response = response['Qualification']
        assert response['Status'] == 'Granted'
        assert response['IntegerValue'] == 1
        
# Assigns qual with `qual_id` to every worker who has completed an assignment for the hit with `hitid`
def assign_qual_for_hit(hitid, qual_id): 
    workers = get_workers_for_hit(hitid)
    print("got workers")
    assign_qual(qual_id, workers)
    print("assigned qual")
    confirm_quals(qual_id, workers)
    print("confirmed qual")

# Download data

In [165]:
from bs4 import BeautifulSoup as bs 
import pprint

def pretty_print(obj):
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(obj)
    pp = None

# Downloads all the assignments completed for `hits` as a list of dictionaries. 
# If a download_path is given, also saves that data as json 
def get_assignment_content(hits, download_path="", should_print=False): 
    all_responses = []
    for hit in hits: 
        hitid = hit['HITId']
        assignments = get_all_assignments(hitid)
        print(hitid)
        print(assignments)
        for a in assignments:
            #print(a)
            a_xml = a['Answer']
            #print(a_xml)
            soup = bs(a_xml, "lxml")
            answers = soup.find_all("answer")
            #print(answers)
            results = {'HITId': hitid}
            for ans in answers: 
                identifier = ans.find('questionidentifier').string
                answer = ans.find('freetext').string
                try: 
                    results[identifier] = json.loads(answer)
                except:
                    results[identifier] = answer
            all_responses.append(results)
    if should_print: 
        pretty_print(all_responses)
    if download_path: 
        with open(download_path, 'w') as outfile: 
            json.dump(all_responses, outfile)
    return all_responses
            

In [166]:
responses = get_assignment_content(hits, download_path='responses.json', should_print=False)
len(responses)

3IH9TRB0FBG6ZNKUDKVO181V2OKI1Q
[]
3909MD9T2ZYL9XCGO0KI5V7XB5WFEF
[]
302U8RURJZIEZ0BJHES5D44SDRKNVZ
[]
3G5RUKN2ECKG2TYGEDN9LMFVMZ19N5
[]
3AJA9FLWSCFXP2E2KJA9QA5P3Y2IFQ
[]
3Q7TKIAPOTR2HJ8CEGCWUQUW8PGLDW
[]
360ZO6N6J10AKBDG2C8987QVH9R9M5
[]
3EHIMLB7F7GOESTV6NMIG6NOGHAH8F
[{'AssignmentId': '3VBEN272MLGVJ0BNQHF71H75KOGGSU', 'WorkerId': 'A1G9Z0CL537336', 'HITId': '3EHIMLB7F7GOESTV6NMIG6NOGHAH8F', 'AssignmentStatus': 'Approved', 'AutoApprovalTime': datetime.datetime(2019, 2, 6, 15, 0, 38, tzinfo=tzlocal()), 'AcceptTime': datetime.datetime(2019, 1, 30, 14, 56, 57, tzinfo=tzlocal()), 'SubmitTime': datetime.datetime(2019, 1, 30, 15, 0, 38, tzinfo=tzlocal()), 'ApprovalTime': datetime.datetime(2019, 1, 31, 10, 30, 50, tzinfo=tzlocal()), 'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>workerId</QuestionIdentifier><FreeText>A1G9Z0CL537336</Free

3LCXHSGDLTNUD2H2UKKHCUQC0VPESW
[{'AssignmentId': '388U7OUMF8IM7H0PXQQZQFFTR5BR0H', 'WorkerId': 'ACUP2HVXGZY46', 'HITId': '3LCXHSGDLTNUD2H2UKKHCUQC0VPESW', 'AssignmentStatus': 'Approved', 'AutoApprovalTime': datetime.datetime(2019, 2, 7, 10, 19, 41, tzinfo=tzlocal()), 'AcceptTime': datetime.datetime(2019, 1, 31, 10, 16, 35, tzinfo=tzlocal()), 'SubmitTime': datetime.datetime(2019, 1, 31, 10, 19, 41, tzinfo=tzlocal()), 'ApprovalTime': datetime.datetime(2019, 1, 31, 10, 30, 50, tzinfo=tzlocal()), 'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>workerId</QuestionIdentifier><FreeText>ACUP2HVXGZY46</FreeText></Answer><Answer><QuestionIdentifier>results</QuestionIdentifier><FreeText>{"inputs":[],"outputs":[["600,464,https://www.dropbox.com/s/ywfqip5r1qjzfhp/certificates_4_26_MACXm5Y4_Yc.png?raw=1:3,2,1,205,17,203,17,202,18,201,22,200,23,2

3LG268AV38NICNHPFVKCMKLGET6ER8
[{'AssignmentId': '3W92K5RLWVYDVV373G1L870NTJU5V4', 'WorkerId': 'ACUP2HVXGZY46', 'HITId': '3LG268AV38NICNHPFVKCMKLGET6ER8', 'AssignmentStatus': 'Approved', 'AutoApprovalTime': datetime.datetime(2019, 2, 6, 2, 1, 49, tzinfo=tzlocal()), 'AcceptTime': datetime.datetime(2019, 1, 30, 2, 0, 4, tzinfo=tzlocal()), 'SubmitTime': datetime.datetime(2019, 1, 30, 2, 1, 49, tzinfo=tzlocal()), 'ApprovalTime': datetime.datetime(2019, 1, 30, 2, 12, 44, tzinfo=tzlocal()), 'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>workerId</QuestionIdentifier><FreeText>ACUP2HVXGZY46</FreeText></Answer><Answer><QuestionIdentifier>results</QuestionIdentifier><FreeText>{"inputs":[],"outputs":[["600,464,https://www.dropbox.com/s/ywfqip5r1qjzfhp/certificates_4_26_MACXm5Y4_Yc.png?raw=1:3,2,1,451,345,451,346,451,348,451,350,451,353,451,

3U74KRR67M2ZM87UCJSQCMG7G64NTJ
[{'AssignmentId': '3NKQQ8O39ZM8MTT0DW9ZAXMWSSIUD9', 'WorkerId': 'A1G9Z0CL537336', 'HITId': '3U74KRR67M2ZM87UCJSQCMG7G64NTJ', 'AssignmentStatus': 'Approved', 'AutoApprovalTime': datetime.datetime(2019, 2, 5, 23, 15, 44, tzinfo=tzlocal()), 'AcceptTime': datetime.datetime(2019, 1, 29, 23, 12, 59, tzinfo=tzlocal()), 'SubmitTime': datetime.datetime(2019, 1, 29, 23, 15, 44, tzinfo=tzlocal()), 'ApprovalTime': datetime.datetime(2019, 1, 30, 1, 21, 54, tzinfo=tzlocal()), 'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>workerId</QuestionIdentifier><FreeText>A1G9Z0CL537336</FreeText></Answer><Answer><QuestionIdentifier>results</QuestionIdentifier><FreeText>{"inputs":[],"outputs":[["0,0,https://www.dropbox.com/s/ywfqip5r1qjzfhp/certificates_4_26_MACXm5Y4_Yc.png?raw=1:","377,600,https://www.dropbox.com/s/b8v7jz8u

31MBOZ6PAO8SCR2YEKHLAKCENP4LCW
[{'AssignmentId': '37TD41K0AIQI2OVIOVJOBQMQMWMCSE', 'WorkerId': 'ACUP2HVXGZY46', 'HITId': '31MBOZ6PAO8SCR2YEKHLAKCENP4LCW', 'AssignmentStatus': 'Approved', 'AutoApprovalTime': datetime.datetime(2019, 2, 6, 1, 59, 39, tzinfo=tzlocal()), 'AcceptTime': datetime.datetime(2019, 1, 30, 1, 57, 7, tzinfo=tzlocal()), 'SubmitTime': datetime.datetime(2019, 1, 30, 1, 59, 39, tzinfo=tzlocal()), 'ApprovalTime': datetime.datetime(2019, 1, 30, 2, 12, 47, tzinfo=tzlocal()), 'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>workerId</QuestionIdentifier><FreeText>ACUP2HVXGZY46</FreeText></Answer><Answer><QuestionIdentifier>results</QuestionIdentifier><FreeText>{"inputs":[],"outputs":[["600,464,https://www.dropbox.com/s/ywfqip5r1qjzfhp/certificates_4_26_MACXm5Y4_Yc.png?raw=1:3,2,1,208,15,208,17,207,18,205,18,205,20,203,20

3SNR5F7R92A5JOE6P8Q38SKPRX7EIO
[]
3TY2U1TEB7R9PFK4LRMUH9CH9KSJJQ
[]
3EHVO81VN5263S9A72D31W0G2BOH1F
[]
3V0TR1NRVAJ8L4YFOEMDI8KF108A4A
[]
3FULMHZ7OUE9N574LHCIRZFFVHXM46
[]
3W1K7D6QSBYT7BRWFLCZ7FHUKDWBZI
[]
3OND0WXMHWW5NT3MJQ8I8GJARKSEHV
[]
3TKXBROM5TRDU40Y391U58XHW53JIE
[]
3FVBZG9CLJV6W3OSBJ93NRH6XJIH0C
[]
36818Z1KV3UN38FBO0HU83S9N0CA3P
[]
3WYZV0QBFJU42GRHE3OEAU1JCDYBXL
[]
36D1BWBEHNIZKRSHH3TUYKIHB31M2A
[]
35JDMRECC4QRNX8BZKQ0NG79CKFEGW
[]
3MJ9GGZYO3NYIQJSK03PPTNHHMCA2E
[{'AssignmentId': '3D3VGR7TA1WIO9ZMGD8ISDD0UGN3RE', 'WorkerId': 'ACUP2HVXGZY46', 'HITId': '3MJ9GGZYO3NYIQJSK03PPTNHHMCA2E', 'AssignmentStatus': 'Approved', 'AutoApprovalTime': datetime.datetime(2019, 2, 5, 5, 6, 5, tzinfo=tzlocal()), 'AcceptTime': datetime.datetime(2019, 1, 29, 5, 4, 15, tzinfo=tzlocal()), 'SubmitTime': datetime.datetime(2019, 1, 29, 5, 6, 5, tzinfo=tzlocal()), 'ApprovalTime': datetime.datetime(2019, 1, 30, 1, 21, 55, tzinfo=tzlocal()), 'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers

3KTCJ4SCVGISV6ALYZTGT18CJXNM1T
[]
3R868ACW4STPYSZ7V74D8Q8R0QOGZ4
[]
3QGHA0EA0JHPDGMR4YSS8BNC3BSBWS
[]
3PKJ68EHDNDDY9PEXRJ95WW2WSOJHL
[]
36GJS3V78V7ZYDU3DL1RKWK1HSBJGM
[]
3U18MJKL1U3I0R5JQXX0NCG3SJZCNI
[]
3UY4PIS8QR2YDOVLA08G6GXCOM11NA
[]
30U1YOGZGADPLW32YP9X6C8R0ICDSO
[]
3XT3KXP24ZF169LRJQ75W4SCAYN6IX
[]
3S37Y8CWI8H5SH1IPJI50XH6TIDW47
[{'AssignmentId': '39GAF6DQWSHE7D2O2TZ95HN6H2A1V1', 'WorkerId': 'ACUP2HVXGZY46', 'HITId': '3S37Y8CWI8H5SH1IPJI50XH6TIDW47', 'AssignmentStatus': 'Approved', 'AutoApprovalTime': datetime.datetime(2019, 2, 1, 15, 11, 44, tzinfo=tzlocal()), 'AcceptTime': datetime.datetime(2019, 1, 25, 15, 10, 6, tzinfo=tzlocal()), 'SubmitTime': datetime.datetime(2019, 1, 25, 15, 11, 44, tzinfo=tzlocal()), 'ApprovalTime': datetime.datetime(2019, 1, 25, 15, 14, 41, tzinfo=tzlocal()), 'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionId

3S8A4GJRD3K6SF5RIX6OBTI6I0HV6H
[{'AssignmentId': '392CY0QWG28QD6F7Q4JSOQ4FLKO4IK', 'WorkerId': 'ACUP2HVXGZY46', 'HITId': '3S8A4GJRD3K6SF5RIX6OBTI6I0HV6H', 'AssignmentStatus': 'Approved', 'AutoApprovalTime': datetime.datetime(2019, 2, 1, 15, 10, 1, tzinfo=tzlocal()), 'AcceptTime': datetime.datetime(2019, 1, 25, 15, 0, 6, tzinfo=tzlocal()), 'SubmitTime': datetime.datetime(2019, 1, 25, 15, 10, 1, tzinfo=tzlocal()), 'ApprovalTime': datetime.datetime(2019, 1, 25, 15, 14, 41, tzinfo=tzlocal()), 'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>workerId</QuestionIdentifier><FreeText>ACUP2HVXGZY46</FreeText></Answer><Answer><QuestionIdentifier>results</QuestionIdentifier><FreeText>{"inputs":[],"outputs":[["600,400,./files/certificates_1_13_MACS70yIlqs.png:3,2,1,250.85,122.5,250.85,123.35,250.85,128.35,250.85,133.35,252.5,138.35,252.5,143.35

3LG268AV38NICNHPFVKCMKLGD5MERB
[]
3M67TQBQQH59IA4R59I4ZGVVM4IA9P
[]
3126F2F5F8KX0A1A23GNHPL1CNYEPJ
[]


13